# Diffusion Monte Carlo Method
## 1次元調和振動子の基底状態
ハミルトニアン

$$
H = -\frac{\hbar^2}{2m}\frac{d^2}{dx^2} + \frac{1}{2}m\omega^2 x^2.
$$

厳密解（$\hbar = m = \omega = 1$）
\begin{align}
    \psi_0(x) &= \pi^{-1/4} e^{-x^2/2}, \\
    E_0 &= \frac{1}{2}.
\end{align}


In [58]:
import numpy as np

def dmc_1d_harmonic(
    n_walkers=2000,
    n_steps=4000,
    dt=0.01,
    burn_in=500,
    alpha=1.0,
    seed=None,
):
    """
    Diffusion Monte Carlo for 1D harmonic oscillator ground state.
    H = -1/2 d^2/dx^2 + 1/2 x^2  (atomic units: ħ = m = ω = 1)

    Importance-sampled DMC with trial Psi_T(x) = exp(-alpha x^2 / 2).
    Local energy: E_L(x) = alpha/2 + (1 - alpha**2) * x**2 / 2
      -> alpha=1.0 なら E_L(x) ≡ 0.5（分散ゼロの理想ケース）

    Parameters
    ----------
    n_walkers : 初期ウォーカー数（目標個体数）
    n_steps   : 総ステップ数
    dt        : 時間刻み Δτ
    burn_in   : 熱化のための前半ステップ数（推定から除外）
    alpha     : 試行関数パラメータ（=1.0 で厳密基底を再現） guiding function Psi_Gに含まれるパラメータで、本来はVMCによって得られる
    seed      : 乱数シード

    Returns
    -------
    E_mean    : 推定エネルギー（混合推定量）の平均
    E_sem     : 標準誤差（標本標準偏差/√サンプル数）
    history   : 各ステップの混合推定量（burn-in 以降）
    """
    rng = np.random.default_rng(seed)
    # LATER: テキストではVMCで得られた分布を使う
    # 初期分布は適当に広がった正規分布
    x = rng.normal(loc=0.0, scale=1.0, size=n_walkers).astype(np.float64)

    # Trial energy, populationを制御するための値で出来るだけ求めたい基底エネルギーに近い値をセットする
    # 0.5をセットすればよいが、すでに厳密値と一致しているため、あえて少しずらした値を入れておく
    # LATER: 本来はVMCで求めたエネルギー期待値を入れる
    E_T = 0.3

    # ローカルエネルギー（重要サンプリング系）
    # E_L(x) = Psi_G^(-1) * H * Psi_G = alpha/2 + (1 - alpha^2) * x^2 / 2
    def local_energy(x):
        return 0.5 * alpha + 0.5 * (1.0 - alpha**2) * (x * x)

    # TODO: なんでこうなる？？
    # ドリフト（量子力） F = ∇ ln Psi_T^2 = -2 alpha x（1D）
    def F(x):
        return -2.0 * alpha * x

    E_samples = []

    for step in range(n_steps):
        # ---- drifted diffusion proposal ----
        # x' = x + (dt/2) F(x) + χ, with χ ~ N(0, dt)
        x_prop = x + 0.5 * dt * F(x) + rng.normal(0.0, np.sqrt(dt), size=x.shape)

        # ローカルエネルギー（対称化のため x と x' の平均を使う）
        EL_x = local_energy(x)
        EL_p = local_energy(x_prop)

        # ---- branching weights ----
        # w = exp( -dt * ( (E_L(x)+E_L(x'))/2 - E_T ) )
        # TODO
        w = np.exp(-dt * (0.5 * (EL_x + EL_p) - E_T))

        # 確率的丸め（stochastic rounding）で整数子孫数へ
        m = np.floor(w + rng.random(w.size)).astype(int)

        # 子孫を生成
        # TODO
        survivors_mask = m > 0
        # if not np.any(survivors_mask):
        #     # 全滅回避：リセット（まれに起きる極端な揺らぎへの保険）
        #     x = rng.normal(0.0, 1.0, size=target_N)
        #     E_T = 0.5
        #     continue

        # TODO
        x_new = np.repeat(x_prop[survivors_mask], m[survivors_mask])

        # ---- population control ----
        N_new = x_new.size
        # 参照エネルギーをログ比で調整（定番の簡便式）
        # E_T ← E_T + (1/Δτ) * ln(N_target / N_new)
        # TODO
        E_T = E_T + (1.0 / dt) * np.log(max(1, n_walkers) / N_new)

        # 人口が過度に膨張・縮小した場合、薄いリサンプリングで安定化
        # （個体値の統計は保ちやすい多重サンプリング方式）
        # if N_new > 4 * target_N:
        #     choose = rng.choice(N_new, size=2 * target_N, replace=False)
        #     x = x_new[choose]
        # elif N_new < target_N // 4:
        #     # 足りなければリサンプルで補充
        #     choose = rng.choice(N_new, size=target_N, replace=True)
        #     x = x_new[choose]
        # else:
        #     x = x_new

        # ---- mixed estimator: エネルギー推定（burn-in 後に記録）----
        # TODO
        if step >= burn_in:
            # 混合推定量は単純に E_L のウォーカー平均
            E_step = np.mean(local_energy(x))
            E_samples.append(E_step)

    if len(E_samples) == 0:
        return np.nan, np.nan, np.array([])

    # TODO: どうやって計算してる？
    E_arr = np.array(E_samples)
    E_mean = E_arr.mean()
    # 有効サンプル間の相関を無視した素朴な標準誤差（目安）
    E_sem = E_arr.std(ddof=1) / np.sqrt(len(E_arr))

    return E_mean, E_sem, E_arr


E, dE, trace = dmc_1d_harmonic()
print(f"DMC estimate: E0 = {E:.6f} ± {dE:.6f}  (exact = 0.5)")

DMC estimate: E0 = 0.500000 ± 0.000000  (exact = 0.5)


In [18]:
# ChatGPTで生成（テキストのアルゴリズムに従った実装）

# import numpy as np
# import matplotlib.pyplot as plt

# # --- 1D Harmonic Oscillator parameters ---
# omega = 1.0
# m = 1.0
# dt = 0.01         # time step
# Nw = 2000         # number of walkers
# nsteps = 500      # number of iterations
# rng = np.random.default_rng(42)

# # Guiding function: Gaussian (Ψ_G ∝ e^{-x²/2})
# def psi_G(x):
#     return np.exp(-0.5 * x**2)

# def drift(x):
#     # Drift term = ∇ ln Ψ_G = -x
#     return -x

# def local_energy(x):
#     # Local energy for Ψ_G = e^{-x²/2} is exact for harmonic oscillator
#     return 0.5 * (x**2 - 1.0)

# # --- Initialization ---
# walkers = rng.normal(0, 1/np.sqrt(2), Nw)
# ET = 0.5  # initial trial energy
# energies = []

# for step in range(nsteps):
#     # Drift-diffusion step (Langevin move)
#     drift_term = drift(walkers)
#     random_displacement = rng.normal(0, np.sqrt(dt), Nw)
#     new_walkers = walkers + drift_term * dt + random_displacement

#     # Local energies
#     EL_old = local_energy(walkers)
#     EL_new = local_energy(new_walkers)

#     # Branching factor m (Eq. 24.24)
#     m = np.exp(-0.5 * dt * (EL_new + EL_old - 2 * ET))

#     # Population control: stochastic rounding
#     u = rng.random(Nw)
#     copies = np.floor(m + u).astype(int)
#     new_population = []
#     for i, ncopy in enumerate(copies):
#         if ncopy > 0:
#             new_population += [new_walkers[i]] * ncopy

#     walkers = np.array(new_population)
#     if len(walkers) == 0:
#         raise RuntimeError("All walkers died; reduce dt or adjust ET.")

#     # Compute average energy and adjust ET
#     E0 = np.mean(local_energy(walkers))
#     ET = E0 - (np.log(len(walkers) / Nw)) / dt  # population control

#     energies.append(E0)

# # --- Results ---
# print(f"Estimated ground-state energy ≈ {np.mean(energies[int(nsteps/2):]):.5f}")
# print(f"Exact value = 0.5")

# plt.plot(energies, label="DMC energy estimate")
# plt.axhline(0.5, color="r", linestyle="--", label="Exact E0 = 0.5")
# plt.xlabel("Iteration")
# plt.ylabel("Energy")
# plt.legend()
# plt.show()


ValueError: operands could not be broadcast together with shapes (2021,) (2000,) 